In [0]:
# ============================================
# Configuration
# ============================================

CATALOG = "fraud_detection"
SCHEMA = "raw"

# Table names
SILVER_TABLE = f"{CATALOG}.{SCHEMA}.silver_all_transactions"
GOLD_ALERTS_TABLE = f"{CATALOG}.{SCHEMA}.gold_fraud_alerts"
METRICS_TABLE = f"{CATALOG}.{SCHEMA}.gold_fraud_metrics_realtime"
PERFORMANCE_TABLE = f"{CATALOG}.{SCHEMA}.gold_system_performance"

print("✅ Config loaded")


✅ Config loaded


In [0]:
# ============================================
# BEFORE Optimization - Current Stats
# ============================================

print("📊 CURRENT TABLE STATISTICS (BEFORE OPTIMIZATION)")
print("=" * 70)

# Silver table stats
silver_count = spark.table(SILVER_TABLE).count()
silver_files = spark.sql(f"DESCRIBE DETAIL {SILVER_TABLE}").select("numFiles").first()[0]
silver_size = spark.sql(f"DESCRIBE DETAIL {SILVER_TABLE}").select("sizeInBytes").first()[0]

print(f"\n🥈 Silver Table: {SILVER_TABLE}")
print(f"   Records: {silver_count:,}")
print(f"   Files: {silver_files}")
print(f"   Size: {silver_size / 1024 / 1024:.2f} MB")

# Gold alerts stats
gold_count = spark.table(GOLD_ALERTS_TABLE).count()
gold_files = spark.sql(f"DESCRIBE DETAIL {GOLD_ALERTS_TABLE}").select("numFiles").first()[0]
gold_size = spark.sql(f"DESCRIBE DETAIL {GOLD_ALERTS_TABLE}").select("sizeInBytes").first()[0]

print(f"\n🥇 Gold Alerts Table: {GOLD_ALERTS_TABLE}")
print(f"   Records: {gold_count:,}")
print(f"   Files: {gold_files}")
print(f"   Size: {gold_size / 1024 / 1024:.2f} MB")

print(f"\n⚠️  Many small files = slow queries!")
print(f"   Target: < 5 files per table for best performance")


📊 CURRENT TABLE STATISTICS (BEFORE OPTIMIZATION)

🥈 Silver Table: fraud_detection.raw.silver_all_transactions
   Records: 1,110
   Files: 19
   Size: 0.09 MB

🥇 Gold Alerts Table: fraud_detection.raw.gold_fraud_alerts
   Records: 29
   Files: 27
   Size: 0.10 MB

⚠️  Many small files = slow queries!
   Target: < 5 files per table for best performance


In [0]:
# ============================================
# Optimize Silver Table with Z-Ordering
# ============================================

print("\n🔧 OPTIMIZING SILVER TABLE...")
print("=" * 70)

import time
start_time = time.time()

# Optimize and Z-Order by frequently queried columns
spark.sql(f"""
    OPTIMIZE {SILVER_TABLE}
    ZORDER BY (card_id, ingest_ts)
""")

elapsed = time.time() - start_time

print(f"✅ Silver table optimized in {elapsed:.1f} seconds")
print(f"   - Compacted small files into larger ones")
print(f"   - Z-Ordered by card_id and ingest_ts for faster lookups")

# Check AFTER stats
silver_files_after = spark.sql(f"DESCRIBE DETAIL {SILVER_TABLE}").select("numFiles").first()[0]
print(f"\n📊 Files BEFORE: {silver_files} → AFTER: {silver_files_after}")
print(f"   File reduction: {((silver_files - silver_files_after) / silver_files * 100):.1f}%")



🔧 OPTIMIZING SILVER TABLE...
✅ Silver table optimized in 8.8 seconds
   - Compacted small files into larger ones
   - Z-Ordered by card_id and ingest_ts for faster lookups

📊 Files BEFORE: 19 → AFTER: 1
   File reduction: 94.7%


In [0]:
# ============================================
# Optimize Gold Alerts Table with Z-Ordering
# ============================================

print("\n🔧 OPTIMIZING GOLD ALERTS TABLE...")
print("=" * 70)

start_time = time.time()

# Optimize and Z-Order by frequently filtered columns
spark.sql(f"""
    OPTIMIZE {GOLD_ALERTS_TABLE}
    ZORDER BY (card_id, alert_timestamp, severity)
""")

elapsed = time.time() - start_time

print(f"✅ Gold alerts optimized in {elapsed:.1f} seconds")

# Check AFTER stats
gold_files_after = spark.sql(f"DESCRIBE DETAIL {GOLD_ALERTS_TABLE}").select("numFiles").first()[0]
print(f"\n📊 Files BEFORE: {gold_files} → AFTER: {gold_files_after}")
print(f"   File reduction: {((gold_files - gold_files_after) / gold_files * 100):.1f}%")



🔧 OPTIMIZING GOLD ALERTS TABLE...
✅ Gold alerts optimized in 4.7 seconds

📊 Files BEFORE: 27 → AFTER: 1
   File reduction: 96.3%


In [0]:
# ============================================
# Analyze Table Statistics for Query Optimization
# ============================================

print("\n📊 ANALYZING TABLE STATISTICS...")
print("=" * 70)

tables = [SILVER_TABLE, GOLD_ALERTS_TABLE, METRICS_TABLE, PERFORMANCE_TABLE]

for table in tables:
    print(f"\nAnalyzing {table}...")
    spark.sql(f"ANALYZE TABLE {table} COMPUTE STATISTICS FOR ALL COLUMNS")
    print(f"✅ {table} analyzed")

print("\n✅ ALL TABLES ANALYZED")
print("   Query optimizer can now make better decisions!")



📊 ANALYZING TABLE STATISTICS...

Analyzing fraud_detection.raw.silver_all_transactions...
✅ fraud_detection.raw.silver_all_transactions analyzed

Analyzing fraud_detection.raw.gold_fraud_alerts...
✅ fraud_detection.raw.gold_fraud_alerts analyzed

Analyzing fraud_detection.raw.gold_fraud_metrics_realtime...
✅ fraud_detection.raw.gold_fraud_metrics_realtime analyzed

Analyzing fraud_detection.raw.gold_system_performance...
✅ fraud_detection.raw.gold_system_performance analyzed

✅ ALL TABLES ANALYZED
   Query optimizer can now make better decisions!


In [0]:
# ============================================
# Vacuum Old Files (Remove deleted data)
# ============================================

print("\n🧹 VACUUMING OLD FILES...")
print("=" * 70)
print("⚠️  This removes old file versions to save storage costs")
print("   Retention: 7 days (168 hours)")

RETENTION_HOURS = 168

for table in [SILVER_TABLE, GOLD_ALERTS_TABLE]:
    print(f"\nVacuuming {table}...")
    spark.sql(f"VACUUM {table} RETAIN {RETENTION_HOURS} HOURS")
    print(f"✅ {table} vacuumed")

print("\n✅ VACUUM COMPLETE - Storage costs reduced!")



🧹 VACUUMING OLD FILES...
⚠️  This removes old file versions to save storage costs
   Retention: 7 days (168 hours)

Vacuuming fraud_detection.raw.silver_all_transactions...
✅ fraud_detection.raw.silver_all_transactions vacuumed

Vacuuming fraud_detection.raw.gold_fraud_alerts...
✅ fraud_detection.raw.gold_fraud_alerts vacuumed

✅ VACUUM COMPLETE - Storage costs reduced!


In [0]:
# ============================================
# Test Query Performance (AFTER optimization)
# ============================================

print("\n⚡ TESTING QUERY PERFORMANCE...")
print("=" * 70)

import time

# Test query 1: Find all fraud for a specific card
test_card_id = spark.table(GOLD_ALERTS_TABLE).select("card_id").first()[0]

start_time = time.time()
result = spark.sql(f"""
    SELECT * 
    FROM {GOLD_ALERTS_TABLE}
    WHERE card_id = {test_card_id}
    ORDER BY alert_timestamp DESC
""").count()
elapsed_ms = (time.time() - start_time) * 1000

print(f"\nQuery 1: Find fraud for card {test_card_id}")
print(f"   Results: {result} rows")
print(f"   Time: {elapsed_ms:.0f}ms")
print(f"   Status: {'✅ FAST' if elapsed_ms < 1000 else '⚠️ SLOW'}")

# Test query 2: Recent fraud alerts
start_time = time.time()
result = spark.sql(f"""
    SELECT * 
    FROM {GOLD_ALERTS_TABLE}
    WHERE alert_timestamp >= CURRENT_TIMESTAMP() - INTERVAL 1 HOUR
    ORDER BY alert_timestamp DESC
""").count()
elapsed_ms = (time.time() - start_time) * 1000

print(f"\nQuery 2: Fraud alerts in last hour")
print(f"   Results: {result} rows")
print(f"   Time: {elapsed_ms:.0f}ms")
print(f"   Status: {'✅ FAST' if elapsed_ms < 1000 else '⚠️ SLOW'}")

print("\n" + "=" * 70)
print("✅ OPTIMIZATION COMPLETE!")
print("\n💡 Schedule this notebook to run daily at 2 AM")



⚡ TESTING QUERY PERFORMANCE...

Query 1: Find fraud for card 1025
   Results: 5 rows
   Time: 1242ms
   Status: ⚠️ SLOW

Query 2: Fraud alerts in last hour
   Results: 0 rows
   Time: 714ms
   Status: ✅ FAST

✅ OPTIMIZATION COMPLETE!

💡 Schedule this notebook to run daily at 2 AM
